In [1]:
# script to generate 20 mechanisms with the top 20 parameters to include

# generate the lists of top parameters

In [26]:
import os
import sys
import copy
import pickle
import subprocess

import numpy as np
import rmgpy.chemkin

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
basedir = '/home/moon/autoscience/autoscience/butane/models/rmg_model'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)




In [4]:
# get most uncertain reactions
covariance_file = '/home/moon/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)
uncorrelated_uncertainties = np.diagonal(Sigma_k)

reaction_indices = np.arange(0, len(reaction_list))
reaction_uncertainty_order = [x for _,x in sorted(zip(uncorrelated_uncertainties, reaction_indices))][::-1]

In [5]:
# get most sensitive reactions
base_delay = 0.14857535
reaction_delay_file = '/home/moon/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/reaction_ignition_delays_raw.npy'
reaction_delays_raw = np.load(reaction_delay_file)
reaction_sensitivities = ((reaction_delays_raw - base_delay) / base_delay) / 0.1

reaction_sensitivity_order = [x for _,x in sorted(zip(np.abs(reaction_sensitivities), reaction_indices))][::-1]


In [6]:
# Calculate Improvement Score
DFT_error = 3.0
improvement_score = np.abs(reaction_sensitivities) * (uncorrelated_uncertainties - DFT_error)
improvement_score_order = [x for _,x in sorted(zip(improvement_score, reaction_indices))][::-1]


In [14]:
# get most uncertain species
species_uncertainty = '/home/moon/autoscience/autoscience/paper/models/base_rmg/thermo_uncertainty.npy'
species_indices = np.arange(0, len(reaction_list))
uncorrelated_species_uncertainties = np.load(species_uncertainty)
species_uncertainty_order = [x for _,x in sorted(zip(uncorrelated_species_uncertainties, species_indices))][::-1]

In [8]:
# get most sensitive species
base_delay = 0.14857535
species_delay_file = '/home/moon/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/species_ignition_delays_raw.npy'
species_delays_raw = np.load(species_delay_file)
species_sensitivities = ((species_delays_raw - base_delay) / base_delay) / 0.1

species_sensitivity_order = [x for _,x in sorted(zip(np.abs(species_sensitivities), species_indices))][::-1]


In [9]:
# Calculate Improvement Score
DFT_error = 3.0
species_improvement_score = np.abs(species_sensitivities) * (uncorrelated_species_uncertainties - DFT_error)
species_improvement_score_order = [x for _,x in sorted(zip(species_improvement_score, species_indices))][::-1]

In [17]:
# concatenate species then reactions to sort total
total_sensitivities = np.concatenate((species_sensitivities, reaction_sensitivities))
total_uncertainties = np.concatenate((uncorrelated_species_uncertainties, uncorrelated_uncertainties))
total_indices = np.arange(0, len(species_list) + len(reaction_list))


total_improvement_score = np.abs(total_sensitivities) * (total_uncertainties - DFT_error)
total_improvement_score_order = [x for _,x in sorted(zip(total_improvement_score, total_indices))][::-1]


# Generate the Mechanisms

In [27]:
save_dir = '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs'
for e in range(0, 51):
    print('compiling top', e)
    # load the base mechanism
    basedir = '/home/moon/autoscience/autoscience/butane/models/rmg_model'
    base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
    dictionary = os.path.join(basedir, 'species_dictionary.txt')
    transport = os.path.join(basedir, 'tran.dat')
    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)

    # Load the new kinetics library
#     DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
    DFT_DIR = "/home/moon/autoscience/autoscience/butane/dft/"
    kinetics_lib = os.path.join(DFT_DIR, 'kinetics', 'kinetics')
    ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
    ark_kinetics_database.load_libraries(kinetics_lib)
    print(f'{len(ark_kinetics_database.libraries[""].entries)} kinetics entries loaded')
    
    # Load the new thermo library
    thermo_lib = os.path.join(DFT_DIR, 'thermo', 'thermo')
    ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
    ark_thermo_database.load_libraries(thermo_lib)
    print(f'{len(ark_thermo_database.libraries["harris_butane"].entries)} thermo entries loaded')
    

    species_change_list = []
    reaction_change_list = []
    for f in range(0, e):
        index = total_improvement_score_order[f]
        if index < len(species_list):
            species_change_list.append(index)
        else:
            reaction_change_list.append(index - len(species_list))
    
    # change relevant kinetics
    total_changed = 0
    for key in ark_kinetics_database.libraries[''].entries.keys():
        entry = ark_kinetics_database.libraries[''].entries[key]
        index = entry.index
        for i in range(0, len(reaction_list)):
            if entry.item.is_isomorphic(reaction_list[i]):
                rmg_rxn_index = i
                break
        else:  
            print(f'could not match reaction index {index}')
            continue

        if rmg_rxn_index in reaction_change_list:
            print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
            reaction_list[rmg_rxn_index].kinetics = entry.data
            total_changed += 1
    print(total_changed, 'kinetics changed')
    
    
    # change the relevant thermo
    sp_changed = 0
    for key in ark_thermo_database.libraries['harris_butane'].entries.keys():
        entry = ark_thermo_database.libraries['harris_butane'].entries[key]

        for i, species in enumerate(species_list):
            if entry.item.smiles == species.smiles:
                rmg_species_index = i
                break
        else:
            entry_sp = rmgpy.species.Species(smiles=entry.item.smiles)
            for j, species in enumerate(species_list):
                if entry_sp.is_isomorphic(species.molecule[0]):
                    rmg_species_index = i
                    'matched'
                    break
            else:
                continue
                raise ValueError (f'could not match species index {entry}')

        if rmg_species_index in species_change_list:
            print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
            species_list[rmg_species_index].thermo = entry.data

            sp_changed += 1
    
    print(sp_changed, 'species changed')
    
    # export the mechanism
    chemkin_file = os.path.join(save_dir, f'top_{e}.inp')
    rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
    subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])
    subprocess.run(['ck2yaml', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.yaml'])

compiling top 0
129 kinetics entries loaded
178 thermo entries loaded
could not match reaction index 2014
could not match reaction index 2474
could not match reaction index 2477
could not match reaction index 2478
could not match reaction index 2479
could not match reaction index 2486
could not match reaction index 2488
could not match reaction index 2489
could not match reaction index 2492
could not match reaction index 2494
could not match reaction index 2495
could not match reaction index 2496
could not match reaction index 2497
could not match reaction index 2499
could not match reaction index 2500
could not match reaction index 2507
could not match reaction index 2566
0 kinetics changed


0 species changed
Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_0.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_0.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
compiling top 1
129 kinetics entries loaded
178 thermo entries loaded
could not match reaction index 2014
could not match reaction index 2474
could not match reaction index 2477
could not match reaction index 2478
could not match reaction index 2479
could not match reaction index 2486
could not match reaction index 2488
could not match reaction index 2489
could not match reaction index 2492
could not match reaction index 2494
could not match reaction index 2495
could not match reaction index 2496
could not match reaction index 2497
could not match reaction index 2499
could not match reacti

1 species changed
Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_1.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_1.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
compiling top 2
129 kinetics entries loaded
178 thermo entries loaded
changing reaction 915 O2(2) + CC(CC[O])OO(777) <=> HO2(16) + NC4KET13(252)
could not match reaction index 2014
could not match reaction index 2474
could not match reaction index 2477
could not match reaction index 2478
could not match reaction index 2479
could not match reaction index 2486
could not match reaction index 2488
could not match reaction index 2489
could not match reaction index 2492
could not match reaction index 2494
could not match reaction index 2495
could not match reaction index 2496
could not match rea

Changing estimated thermo 87: [CH2]C(=O)COO(911)
1 species changed
Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_2.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_2.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
compiling top 3
129 kinetics entries loaded
178 thermo entries loaded
changing reaction 749 HCO(19) + C4H8-1(188) <=> CH2O(9) + C4H71-3(190)
changing reaction 915 O2(2) + CC(CC[O])OO(777) <=> HO2(16) + NC4KET13(252)
could not match reaction index 2014
could not match reaction index 2474
could not match reaction index 2477
could not match reaction index 2478
could not match reaction index 2479
could not match reaction index 2486
could not match reaction index 2488
could not match reaction index 2489
could not match reaction index 2492
could n

1 species changed
Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_3.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_3.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
compiling top 4
129 kinetics entries loaded
178 thermo entries loaded
changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
changing reaction 749 HCO(19) + C4H8-1(188) <=> CH2O(9) + C4H71-3(190)
changing reaction 915 O2(2) + CC(CC[O])OO(777) <=> HO2(16) + NC4KET13(252)
could not match reaction index 2014
could not match reaction index 2474
could not match reaction index 2477
could not match reaction index 2478
could not match reaction index 2479
could not match reaction index 2486
could not match reaction index 2488
could not match reaction index 2489
could not match reaction index

...

could not match reaction index 2497
could not match reaction index 2499
could not match reaction index 2500
could not match reaction index 2507
could not match reaction index 2566
28 kinetics changed
Changing estimated thermo 87: [CH2]C(=O)COO(911)
Changing estimated thermo 90: [O]OCC(=O)COO(928)
Changing estimated thermo 99: [CH2]C1CC1(4343)
Changing estimated thermo 109: [O]OC1CCOOC1(9510)
4 species changed
Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_46.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_46.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
compiling top 47
129 kinetics entries loaded
178 thermo entries loaded
changing reaction 213 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
c

Changing estimated thermo 87: [CH2]C(=O)COO(911)
Changing estimated thermo 90: [O]OCC(=O)COO(928)
Changing estimated thermo 99: [CH2]C1CC1(4343)
Changing estimated thermo 109: [O]OC1CCOOC1(9510)
4 species changed
Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_47.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_47.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
compiling top 48
129 kinetics entries loaded
178 thermo entries loaded
changing reaction 213 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
changing reaction 370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
changing reaction 371 CH3CHO(35) + PC4H9(182) <=> CH2CHO(21) + butane(1)
changing reaction 418 HO2(16) + CH3CO(20) <=> H2O2(17) 

could not match reaction index 2507
could not match reaction index 2566
28 kinetics changed
Changing estimated thermo 87: [CH2]C(=O)COO(911)
Changing estimated thermo 90: [O]OCC(=O)COO(928)
Changing estimated thermo 99: [CH2]C1CC1(4343)
Changing estimated thermo 109: [O]OC1CCOOC1(9510)
4 species changed
Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_48.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_48.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
compiling top 49
129 kinetics entries loaded
178 thermo entries loaded
changing reaction 213 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
changing reaction 370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
changing reaction 371 CH3CHO(35) + P

Changing estimated thermo 87: [CH2]C(=O)COO(911)
Changing estimated thermo 90: [O]OCC(=O)COO(928)
Changing estimated thermo 99: [CH2]C1CC1(4343)
Changing estimated thermo 109: [O]OC1CCOOC1(9510)
4 species changed
Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_49.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_49.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
compiling top 50
129 kinetics entries loaded
178 thermo entries loaded
changing reaction 213 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
changing reaction 370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
changing reaction 371 CH3CHO(35) + PC4H9(182) <=> CH2CHO(21) + butane(1)
changing reaction 418 HO2(16) + CH3CO(20) <=> H2O2(17) 

Wrote CTI mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_50.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
Wrote YAML mechanism file to '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs/top_50.yaml'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...
PASSED
